### Import Libraries

In [1]:
from requests_html import HTMLSession
from bs4 import BeautifulSoup
import pandas as pd
from tqdm import tqdm_notebook as tqdm
import re
import math

### Fuctions to extract info from CORE webpage

In [2]:
# this function for the getting information from the web page

def get_paperinfo(paper_url):

    #download the page
    session = HTMLSession()
    response=session.get(paper_url)

  # check successful response
    if response.status_code != 200:
        print('Status code:', response.status_code)
        raise Exception('Failed to fetch web page ')

  #parse using beautiful soup
    paper_doc = BeautifulSoup(response.text,'html.parser')

    return paper_doc


def core_attributes(doc):
    
    '''
    recieves a doc file from the get_paperinfo function
    and returns title, ref and abstract from the CORE webpage
    '''
    
    # text contains title and references
    text = doc.main.find_all('h3')
    # ab contains the abstracts
    ab = doc.find_all("div", itemprop="abstract")
    
    papers = []
  
    for i, (t,a) in enumerate(zip(text,ab)):
        title = t.get_text()
        ref = t.a.get('href')
        abstract = a.get_text()
        
        # check if the article does not have an abstract
        if abstract == '':
            print(f'Article{i}: {title} has an empty abstract')
        
        # else append to a list
        else:
            papers.append([title, ref, abstract])

    return papers


### Generation of pandas df with stored paper data

In [10]:
# Getting information from CORE
gene = 'rab39b'
doc = get_paperinfo(f'https://core.ac.uk/search?q={gene}+AND+language%3A"en"&page=1')
n_output = doc.find("div", class_="styles_header__dGlUR").p.get_text()
#n_output = n_output.p.get_text()
number = re.findall('[-+]?[.]?[\d]+(?:,\d\d\d)*[\.]?\d*(?:[eE][-+]?\d+)?', n_output)
number = float(number[0].replace(',', ''))
round_ = math.ceil(number/10)

#Create empty dataframe
df = pd.DataFrame(columns = ['title', 'ref', 'abstract'])

#Iterate over different pages on the CORE database to obtain titlt, ref and abstract and add to df
for l in tqdm(range(1,round_)):
    doc = get_paperinfo(f'https://core.ac.uk/search?q={gene}+AND+language%3A%22en%22&page={l}')
    papers = core_attributes(doc)
    dft = pd.DataFrame(papers, columns = ['title', 'ref', 'abstract'])
    df = pd.concat([df,dft])  
    
#df.to_csv('paper_info.csv')

/var/folders/_x/tfg8s2ks4n1ftkkwzp5sqjpc0000gn/T/ipykernel_1252/238930726.py:14: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  for l in tqdm(range(1,round_)):


  0%|          | 0/12 [00:00<?, ?it/s]

Article6: Parkinson’s:A Disease of Aberrant Vesicle Trafficking has an empty abstract
Article4: Characterization of Parkinson's Disease-associated LRRK2 Kinase has an empty abstract
Article0: Integrated Stress Response Activity Marks Stem Cells in Normal Hematopoiesis and Leukemia has an empty abstract
Article4: Comprehensive knockout analysis of the Rab family GTPases in epithelial cells has an empty abstract
Article2: Genetics dissection and functional studies in Hirschsprung disease has an empty abstract
Article5: Features of structural binding motifs and their predictive power has an empty abstract
Article8: BCORL1 regulates stemness in hepatoblastoma cells through inhibition of KRT19 expression has an empty abstract
Article9: BCORL1 regulates stemness in hepatoblastoma cells through inhibition of KRT19 expression has an empty abstract
Article5: Multi-omics analysis of human brain tissue and an animal model of Parkinson’s Disease has an empty abstract
Article6: Deletion of Setdb1 i

In [11]:
df

,title,ref,abstract
0,The late endocytic Rab39a GTPase regulates the...,https://core.ac.uk/works/40912718,"Given their obligate intracellular lifestyle, ..."
1,Twist1 Directly Regulates Genes That Promote C...,https://core.ac.uk/works/4005785,"Twist1, a basic helix-loop-helix transcription..."
2,Rab39a and Rab39b display different intracellu...,https://core.ac.uk/works/41052356,Rab GTPases define the identity and destiny of...
3,Effect of Rab39 on Autophagy,https://core.ac.uk/works/86243525,Rab蛋白属于Ras超家族，是其中最大的一个家族。在哺乳动物细胞中已经发现70种以上的Rab...
4,GM-CSF Signalling Boosts Dramatically IL-1Prod...,https://core.ac.uk/works/3954955,GM-CSF is mostly known for its capacity to pro...
...,...,...,...
4,Analysis of Helicobacter pylori VacA-containin...,https://core.ac.uk/works/62680127,The human pathogen Helicobacter pylori coloniz...
5,Investigating the Nuclear Function of the C9or...,https://core.ac.uk/works/8608392,Amyotrophic lateral sclerosis (ALS) is a termi...
6,The Investigation of Targets for Therapy in Br...,https://core.ac.uk/works/4764206,Gliomas are among the most difficult of tumour...
7,Autism Spectrum Disorders,https://core.ac.uk/works/126151060,Estimated prevalence rates of autism spectrum ...


#### Here comes the difficult part..... how to extract info from the abstracts..
One way to do it is to tokenize the texts

In [32]:
from nltk.stem import PorterStemmer
from nltk.tokenize import word_tokenize
ps = PorterStemmer()
#proceses = ['lysosome', 'endocytosis', 'secretion', 'lipidation', 'kinase', 'glycosilation', 'heparan sulfate']

abstract_list = []
for txt in df['abstract']:
    text = txt
    tokens = word_tokenize(text)
    stemmed = []
    for token in tokens:
        stemmed_word = ps.stem(token)
        stemmed.append(stemmed_word)
    abstract_list.append(stemmed)
print(abstract_list[0])

['given', 'their', 'oblig', 'intracellular', 'lifestyl', ',', 'chlamydia', 'trachomati', 'ensur', 'their', 'access', 'to', 'multipl', 'host', 'sourc', 'of', 'essenti', 'lipid', 'by', 'interf', 'vesicular', 'transport', '.', 'these', 'bacteria', 'hijack', 'rab6-', ',', 'rab11-', 'and', 'rab14-control', 'traffick', 'pathway', 'to', 'acquir', 'sphingomyelin', 'from', 'the', 'golgi', 'apparatu', '.', 'anoth', 'import', 'sourc', 'of', 'sphingolipid', ',', 'phospholipid', 'and', 'cholesterol', 'are', 'multivesicular', 'bodi', '(', 'mvb', ')', '.', 'despit', 'their', 'particip', 'in', 'chlamydi', 'inclus', 'develop', 'and', 'bacteri', 'replic', ',', 'the', 'molecular', 'mechan', 'mediat', 'mvbs-inclus', 'interact', 'remain', 'unknown', '.', 'in', 'the', 'present', 'studi', ',', 'we', 'demonstr', 'that', 'rab39a', 'label', 'a', 'subset', 'of', 'late', 'endocyt', 'vesicl', '-mainli', 'mvbs-', 'that', 'move', 'along', 'microtubul', '.', 'moreov', ',', 'rab39a', 'is', 'activ', 'recruit', 'to', 'c

In [33]:
new_abstract_list = [' '.join(abstract) for abstract in abstract_list]
print(new_abstract_list[0])

given their oblig intracellular lifestyl , chlamydia trachomati ensur their access to multipl host sourc of essenti lipid by interf vesicular transport . these bacteria hijack rab6- , rab11- and rab14-control traffick pathway to acquir sphingomyelin from the golgi apparatu . anoth import sourc of sphingolipid , phospholipid and cholesterol are multivesicular bodi ( mvb ) . despit their particip in chlamydi inclus develop and bacteri replic , the molecular mechan mediat mvbs-inclus interact remain unknown . in the present studi , we demonstr that rab39a label a subset of late endocyt vesicl -mainli mvbs- that move along microtubul . moreov , rab39a is activ recruit to chlamydi inclus throughout the pathogen life cycl by a bacterial-driven process and depend on it gtp/gdp bind state . interestingli , rab39a particip in the deliveri of mvb and host sphingolipid to matur chlamydi inclus therebi promot inclus growth and bacteri develop . altogeth , our find indic that rab39a favour chlamydi

In [89]:
from sklearn.feature_extraction.text import CountVectorizer

# initialize
cv = CountVectorizer(max_features=100,stop_words='english') 
cv_matrix = cv.fit_transform(new_abstract_list) 
# create document term matrix
df_dtm = pd.DataFrame(cv_matrix.toarray(), index=df['title'].values, columns=cv.get_feature_names_out())
df_dtm

,43,activ,addit,affect,aggreg,al,analysi,associ,bind,c9orf72,...,tdp,th17,thi,transcript,transport,understand,use,variant,wa,work
The late endocytic Rab39a GTPase regulates the interaction between multivesicular bodies and chlamydial inclusions.,0,1,0,0,0,0,0,0,1,0,...,0,0,1,0,1,0,0,0,0,0
Twist1 Directly Regulates Genes That Promote Cell Proliferation and Migration in Developing Heart Valves,0,0,1,0,0,0,0,0,3,0,...,0,0,0,3,0,0,1,0,2,0
Rab39a and Rab39b display different intracellular distribution and function in sphingolipids and phospholipids transport,0,0,0,0,0,4,1,0,0,0,...,0,0,0,0,5,0,2,0,0,0
Effect of Rab39 on Autophagy,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,1,0,0,0,0,0
GM-CSF Signalling Boosts Dramatically IL-1Production,0,0,1,0,0,0,0,0,0,0,...,0,0,1,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
Analysis of Helicobacter pylori VacA-containing vacuoles and VacA intracellular trafficking,0,2,0,0,0,0,0,0,0,0,...,0,0,6,0,2,0,1,0,3,0
Investigating the Nuclear Function of the C9orf72 Protein in Amyotrophic Lateral Sclerosis,0,0,0,0,0,8,0,0,1,20,...,0,0,3,0,0,1,0,0,2,1
The Investigation of Targets for Therapy in Brain Tumours,0,0,0,0,0,0,3,0,0,0,...,0,0,0,0,0,0,1,0,3,0
Autism Spectrum Disorders,0,0,3,2,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [79]:
df_dtm['will'].sum()

16

In [56]:
df_dtm

,005,05,0cnp,10,100,11,110,111,11a,11b,...,在细胞内蛋白质运输过程中扮演非常重要的角色,学位,是其中最大的一个家族,本文通过对rab39的定位研究发现,理学硕士院系专业,而rab39b基因定位于性染色体xq28上,药学院_化学生物学学号,进而调控细胞囊泡的形成,靶向,黏附和融合
The late endocytic Rab39a GTPase regulates the interaction between multivesicular bodies and chlamydial inclusions.,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
Twist1 Directly Regulates Genes That Promote Cell Proliferation and Migration in Developing Heart Valves,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
Rab39a and Rab39b display different intracellular distribution and function in sphingolipids and phospholipids transport,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
Effect of Rab39 on Autophagy,0,0,0,0,0,0,0,0,0,0,...,1,1,1,1,1,1,1,1,1,1
GM-CSF Signalling Boosts Dramatically IL-1Production,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
Analysis of Helicobacter pylori VacA-containing vacuoles and VacA intracellular trafficking,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
Investigating the Nuclear Function of the C9orf72 Protein in Amyotrophic Lateral Sclerosis,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
The Investigation of Targets for Therapy in Brain Tumours,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
Autism Spectrum Disorders,0,0,0,0,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
